In [35]:
import os
import numpy as np
import pandas as pd
import pandas_ta as ta
from pathlib import Path
from datetime import datetime


downloads_path = str(Path.home() / "Downloads")
tv_exp_fn = f'{downloads_path}\\BINANCE_BTCUSDT.P, 15.csv'
tst_fn = f'{downloads_path}\\tst.csv'

N = 4
per = 144
mult = 1.414
modeLag = False
modeFast = False
beta = 0.00503274062721631
alpha = 0.0954202818890498
lag = 17.875
m__x = 0.90457971811095


tv_df = pd.read_csv(tv_exp_fn)
os.remove(tv_exp_fn)
open_price = list(tv_df['open'])
high_price = list(tv_df['high'])
low_price = list(tv_df['low'])
close_price = list(tv_df['close'])

m_src = list(ta.hlc3(tv_df['high'], tv_df['low'], tv_df['close']))
tv__f = list(tv_df['tv__f'])
m_srcdata = m_src
m_trdata = [max([high_price[idx] - low_price[idx], abs(high_price[idx] - close_price[idx-1]), abs(low_price[idx] - close_price[idx-1])]) for idx in range(len(open_price))]
_i = N
_a = alpha
_s = m_srcdata
m__f = [0.0]*len(open_price)
m_m2 = (36 if _i == 9 else 28 if _i == 8 else 21 if _i == 7 else 15 if _i == 6 else 10 if _i == 5 else 6 if _i == 4 else 3 if _i == 3 else 1 if _i == 2 else 0)
m_m3 = (84 if _i == 9 else 56 if _i == 8 else 35 if _i == 7 else 20 if _i == 6 else 10 if _i == 5 else 4 if _i == 4 else 1 if _i == 3 else 0)
m__m4 = (126 if _i == 9 else 70 if _i == 8 else 35 if _i == 7 else 15 if _i == 6 else 5 if _i == 5 else 1 if _i == 4 else 0)
m__m5 = (126 if _i == 9 else 56 if _i == 8 else 21 if _i == 7 else 6 if _i == 6 else 1 if _i == 5 else 0)
m__m6 = (84 if _i == 9 else 28 if _i == 8 else 7 if _i == 7 else 1 if _i == 6 else 0)
m__m7 = (36 if _i == 9 else 8 if _i == 8 else 1 if _i == 7 else 0)
m__m8 = (9 if _i == 9 else 1 if _i == 8 else 0)
m__m9 = 1 if _i == 9 else 0
m__f = [np.power(_a, _i) * _s if _i == 1 else np.power(_a, _i) * _s + _i * m__x * _f[idx-1] - (_m2 * np.power(m__x, 2) * _f[2] if _i >= 2 else 0) + (_m3 * np.power(m__x, 3) * _f[3] if _i >= 3 else 0) - (_m4 * np.power(m__x, 4) * _f[4] if _i >= 4 else 0) + (_m5 * np.power(m__x, 5) * _f[5] if _i >= 5 else 0) - (_m6 * np.power(m__x, 6) * _f[6] if _i >= 6 else 0) + (_m7 * np.power(m__x, 7) * _f[7] if _i >= 7 else 0) - (_m8 * np.power(m__x, 8) * _f[8] if _i >= 8 else 0) + (_m9 * np.power(m__x, 9) * _f[9] if _i == 9 else 0) for idx in range(len(close_price))]








exp_df = pd.DataFrame()
exp_df['tv__f'] = tv__f
exp_df['m__m9'] = m__m9
exp_df['chk'] = [f"=ROUND(A{idx+2},1)=ROUND(B{idx+2},1)" for idx in range(len(open_price))]
exp_df['false_count'] = [f"=COUNTIF(C2:C{len(open_price)}, FALSE)"]+['']*(len(open_price)-1)
exp_df.to_csv(tst_fn, index=False)
os.system(f"start EXCEL.EXE {tst_fn}")


0

In [25]:

print(m_m2)

6
